In [1]:
from langchain_ollama import OllamaLLM

llm = OllamaLLM(temperature = 0, model="gemma3:12b")

response = llm.invoke("겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?")
print(response)

겨울철 추위를 잘 견디는 내한성이 강한 나무는 종류가 다양합니다. 몇 가지 대표적인 나무들을 소개하고, 각 나무의 특징과 함께 심을 때 고려해야 할 사항도 함께 알려드릴게요.

**1. 잣나무 (Pinus koraiensis)**

*   **내한성:** 매우 강함 (영하 30도 이상에서도 생존 가능)
*   **특징:** 잎이 무성하고 수형이 아름다워 조경수로 많이 사용됩니다. 묵직한 잣 열매를 맺습니다.
*   **심을 때 고려사항:** 햇볕이 잘 드는 곳을 좋아하며, 배수가 잘 되는 토양에서 잘 자랍니다.

**2. 소나무 (Pinus densiflora)**

*   **내한성:** 강함 (영하 20도 정도까지 견딤)
*   **특징:** 잎이 빽빽하고 곧게 뻗은 수형이 특징입니다. 한국의 대표적인 소나무 종류입니다.
*   **심을 때 고려사항:** 햇볕이 잘 드는 곳을 좋아하며, 토양은 척박해도 잘 자랍니다.

**3. 곰솔 (Pinus pumila)**

*   **내한성:** 매우 강함 (고산지대에서 자라는 나무로, 극한의 추위에도 강함)
*   **특징:** 잎이 짧고 빽빽하며, 꼬불꼬불한 수형이 독특합니다.
*   **심을 때 고려사항:** 햇볕이 잘 드는 곳을 좋아하며, 배수가 잘 되는 척박한 토양에서도 잘 자랍니다.

**4. 단풍나무 (Acer palmatum)**

*   **내한성:** 강함 (영하 15도 정도까지 견딤)
*   **특징:** 가을에 붉게 물드는 단풍이 아름다운 나무입니다. 다양한 품종이 있습니다.
*   **심을 때 고려사항:** 반그늘에서 잘 자라며, 습기가 있는 비옥한 토양을 좋아합니다.

**5. 느티나무 (Zelkova serrata)**

*   **내한성:** 강함 (영하 15도 정도까지 견딤)
*   **특징:** 잎이 넓고 그늘을 잘 만들어 정원수로 많이 사용됩니다. 튼튼한 수령을 자랑합니다.
*   **심을 때 고려사항:** 햇볕이 잘 드는 곳을 좋아하며, 배수가 잘 되는 토양에서 잘 자랍니다.

*

In [5]:
# PDF 문서 로드 및 텍스트 추출 

from langchain_community.document_loaders import PyPDFLoader

loader = PyPDFLoader("수목관리업무매뉴얼(표지제외).pdf", extract_images=True)
pages = loader.load() 
pages[9].page_content

'수목 관리 업무매뉴얼\n4\n나. 기타 수목 관련 용어\n용  어 설  명 비고\n관목 영산홍, 철쭉과 같이 키가 작은 나무\n관 수 물을 대는 것 또는 물을 주는 작업\n관엽식물 식 물 의  잎 을  관 상 의  대 상 으 로  하 는  식 물 의  총 칭\n교 목 느티나무, 단풍나무, 은행나무와 같이 키가 큰 나무\n군 식 여러 그루의 수목을 한군데 모아서 식재하는 것\n낙엽교목 참 나 무 ,  밤 나 무  등 과  같 이  가 을 에  잎 이  떨 어 져 서  봄 에  새 잎 이  나 는  교 목\n냉 해 생육 기간에 빙점(물이 어는 온도) 이상의 온도에서 나타나는 저온 피해\n농약 식물, 동물, 해충, 곰팡이와 같은 원하지 않는 병원체를 죽이기 위해 \n사용하는 화학물질\n눈 따 기 신초가 발육하기 전에 일부 잎눈을 제거하여 생장을 촉진하는 방법\n다량원소 식물이 생장하는 데 많은 양이 필요한 원소로 탄소, 수소, 산소, 질소, 인, \n칼륨, 칼슘, 마그네슘이 속함\n다육식물 건조 기후나 모래 환경에 적응하기 위하여 다육질의 잎에 물을 저장하고 \n있는 식물\n단 식 한 그루의 수목만을 식재하는 것\n도착도 조경수를 굴취 작업한 후의 상차 작업비와 운반까지의 비용을 말한다\n(도착도 = 나무가격 + 굴취비용 + 상차 비용 + 운반비)\n도포제 나무 전지, 전정, 껍질 벗긴 후 상처 부위에 도포하여 병원균을 예방하고자 \n할 때 필요\n되메우기 필요에 의해서 파두었던 공간(관, 박스 구조물, 기초 등)을 흙 등으로 다시 \n채우는 것\n뗏 장 흙이 붙어 있는 상태로 뿌리째 떠낸 잔디의 조각\n만 경 (덩굴) 머루처럼 줄기나 덩굴손으로 물체에 감기거나, 담쟁이덩굴처럼 흡반으로 \n물체에 붙어 기어오르며 자라는 식물의 총칭\n맹 아 수목의 눈이나 생장점이 활동을 시작하여 새싹이 돋는 것\n멀 칭 지표면을 볏짚, 솔잎, 나무껍질, 우드 칩 등으로 덮는 것'

In [6]:
#문장 임베딩 및 벡터 저장소 생성 
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain_community.embeddings import HuggingFaceEmbeddings
from langchain_community.vectorstores import Chroma

#문서를 문장으로 분리 
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size = 1000, 
    chunk_overlap=100,
)

docs = text_splitter.split_documents(pages)

#문장을 임베딩으로 변환하고 벡터 저장소에 저장 
embeddings = HuggingFaceEmbeddings(
    model_name = 'jhgan/ko-sroberta-nli',
    model_kwargs = {'device': 'cpu'},
    encode_kwargs = {'normalize_embeddings':True},
)

vectorstore = Chroma.from_documents(docs, embeddings)

C:\Users\Bistelligence\AppData\Local\Temp\ipykernel_32708\2109326840.py:15: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the `langchain-huggingface package and should be used instead. To use it run `pip install -U `langchain-huggingface` and import as `from `langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\Bistelligence\AppData\Local\pypoetry\Cache\virtualenvs\ollama-app-PvdVXOUT-py3.11\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading weights: 100%|██████████| 199/199 [00:00<00:00, 807.13it/s, Materializing param=pooler.dense.weight]                               
RobertaModel LOAD REPORT from: jhgan/ko-sroberta-nli
Key                 

In [8]:
#검색 쿼리 
query = "겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?"

#가장 유사도가 높은 문장을 하나만 추출 
retriever = vectorstore.as_retriever(search_kwargs={'k':5})

docs = retriever.invoke(query)

print(len(docs))
print(docs[0].page_content)
print(docs[0].metadata)

5
나) 수종별 내한성에 따른 전정 
  추위를 잘 견디어 내는 나무의 성질을 내한성이라고 한다. 생장 중인 잎이나 가는 가지 
등에서 일어나는 상해(霜害)에 대한 저항성을 포함하는 경우와, 휴면 중인 가지나 엄동기
{'page': 137, 'producer': 'Acrobat Distiller 10.1.16 (Windows)', 'creationdate': '2022-12-23T14:12:34+09:00', 'creator': 'PScript5.dll Version 5.2.2', 'moddate': '2022-12-26T18:09:18+09:00', 'author': 'YoungJa70', 'title': '<283232313231352920BCF6B8F1B0FCB8AE20BEF7B9ABB8C5B4BABEF32E687770>', 'total_pages': 207, 'page_label': '138', 'source': '수목관리업무매뉴얼(표지제외).pdf'}


In [9]:
print(docs[1].page_content)
print(docs[1].metadata)

(1) 내한성은 동절기 중 극히 낮은 온도에 견디는 능력이다. 
(가) 내한성이 강한 수종은 한대림에서 자라는 수종으로 자작나무, 오리나무, 사
시나무, 버드나무류, 소나무, 잣나무, 전나무 등이 해당된다. 
(나) 내한성이 약한 수종은 삼나무, 편백, 금송, 히말라야시다, 배롱나무, 파라칸
타, 동백나무, 후박나무, 먼나무 등 주로 남부 지역에서 자라는 수종과 자목
련, 사철나무, 가이즈까향나무, 능소화, 벽오동, 오동나무 등이다. 
(다) 내한성이 약한 수종은 수간을 볏짚이나 새끼 끈으로 싸 주고, 상열을 막기
{'page': 108, 'page_label': '109', 'moddate': '2022-12-26T18:09:18+09:00', 'source': '수목관리업무매뉴얼(표지제외).pdf', 'producer': 'Acrobat Distiller 10.1.16 (Windows)', 'creator': 'PScript5.dll Version 5.2.2', 'total_pages': 207, 'creationdate': '2022-12-23T14:12:34+09:00', 'author': 'YoungJa70', 'title': '<283232313231352920BCF6B8F1B0FCB8AE20BEF7B9ABB8C5B4BABEF32E687770>'}


In [12]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_ollama import OllamaLLM

# LLM
model = OllamaLLM(
    model="gemma3:12b",
    temperature=0
)

#prompt
template = '''Answer the question based only on the following context in Korean :
{context}

Question: {question}'''

prompt = PromptTemplate.from_template(template)

def format_docs(docs):
    return '\n\n'.join([d.page_content for d in docs])

#Rag chain 연결 
rag_chain = ({'context': retriever | format_docs, 'question' : RunnablePassthrough()}
             | prompt
             | model 
             | StrOutputParser())

# Chain 실행 
query = "겨울철에 내한성이 강한 나무에는 어떤 것이 있을까요?"
rag_chain.invoke(query)



'According to the provided text, trees with strong cold resistance (내한성이 강한 수종) include: 자작나무 (birch), 오리나무 (alders), 사시나무 (maples), 버드나무류 (willow trees), 소나무 (pine trees), 잣나무 (Korean pine), and 전나무 (fir trees).'

In [13]:
query = "겨울철에 추위에 약한 나무에는 어떤 것이 있을까요?"
rag_chain.invoke(query)

'According to the provided text, trees that are weak to cold (내한성이 약한 수종) include: 삼나무 (cypress), 편백 (cedar), 금송 (golden larch), 히말라야시다 (Himalayan cedar), 배롱나무 (crape myrtle), 파라칸타 (Parakanta), 동백나무 (camellia), 후박나무 (Japanese evergreen magnolia), 먼나무 (Japanese persimmon), 자목련 (Japanese magnolia), 사철나무 (Japanese holly), 가이즈까향나무 (Katsura tree), 능소화 (trumpet vine), 벽오동 (Korean smoketree), 오동나무 (Chinese paulownia).'